In [ ]:
pip install neo4j py2neo

In [ ]:
pip install pdfplumber

In [6]:
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
import pdfplumber

In [3]:
# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

In [4]:
# Connessione al database
graph = Graph(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [7]:
file_path = "./Hackapizza Dataset/Misc/Manuale di Cucina.pdf"

In [9]:
def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Leggiamo il PDF
pdf_text = extract_text_from_pdf(file_path)

# Stampiamo un'anteprima
print(pdf_text[:500])  # Stampa i primi 500 caratteri


Introduzione
Io sono il grande Sirius Cosmo, lo chef stellare per eccellenza. Se non mi conoscete ancora, preparatevi:
il mio nome è sinonimo di cucina galattica. In questo manuale vi insegnerò tutto quello che serve per
diventare veri cuochi, da Alpha Centauri fino alla Nebulosa del Granchio (dove, a proposito, non hanno
neanche un crostaceo decente). Impareremo insieme le licenze e le abilità fondamentali per cucinare
nello spazio senza mandare in tilt lʼintero sistema di supporto vitale della


In [ ]:
from py2neo import Node, Relationship
import re

def create_graph(text):
    # Creiamo il nodo del documento
    doc_node = Node("Document", name="Manuale di Cucina", content=text[:1000])  # Salviamo solo un estratto
    graph.create(doc_node)

    # Estraiamo le ricette con un pattern semplice (esempio: titoli di ricette in MAIUSCOLO)
    recipes = re.findall(r"\n([A-Z\s]+)\n", text)
    
    for recipe in recipes[:10]:  # Limitiamo a 10 per esempio
        recipe_node = Node("Recipe", name=recipe.strip())
        graph.create(recipe_node)
        graph.create(Relationship(doc_node, "CONTAINS", recipe_node))

    print("✅ Grafo creato con successo su AuraDB!")

# Creiamo il grafo
create_graph(pdf_text)
